# Evaluation of RAG Using Ragas

In the following notebook we'll explore how to evaluate RAG pipelines using a powerful open-source tool called "Ragas". This will give us tools to evaluate component-wise metrics, as well as end-to-end metrics about the performance of our RAG pipelines.

In the following notebook we'll complete the following tasks:

- 🤝 Breakout Room #1:
  1. Install required libraries
  2. Set Environment Variables
  3. Creating a simple RAG pipeline with [LangChain v0.1.0](https://blog.langchain.dev/langchain-v0-1-0/)
  

- 🤝 Breakout Room #2:
  1. Synthetic Dataset Generation for Evaluation using the [Ragas](https://github.com/explodinggradients/ragas)
  2. Evaluating our pipeline with Ragas
  3. Making Adjustments to our RAG Pipeline
  4. Evaluating our Adjusted pipeline against our baseline
  5. Testing OpenAI's Claim

The only way to get started is to get started - so let's grab our dependencies for the day!

## Motivation

A claim, made by OpenAI, is that their `text-embedding-3-small` is better (generally) than their `text-embedding-ada-002` model.

Here's some passages from their [blog](https://openai.com/blog/new-embedding-models-and-api-updates) about the `text-embedding-3` release:

> `text-embedding-3-small` is our new highly efficient embedding model and provides a significant upgrade over its predecessor, the `text-embedding-ada-002` model...

> **Stronger performance.** Comparing `text-embedding-ada-002` to `text-embedding-3-small`, the average score on a commonly used benchmark for multi-language retrieval ([MIRACL](https://github.com/project-miracl/miracl)) has increased from 31.4% to 44.0%, while the average score on a commonly used benchmark for English tasks ([MTEB](https://github.com/embeddings-benchmark/mteb)) has increased from 61.0% to 62.3%.

Well, with a library like Ragas - we can put that claim to the test!

If what they claim is true - we should see an increase on related metrics by using the new embedding model!

# 🤝 Breakout Room #1

## Task 1: Installing Required Libraries

A reminder that one of the [key features](https://blog.langchain.dev/langchain-v0-1-0/) of LangChain v0.1.0 is the compartmentalization of the various LangChain ecosystem packages!

So let's begin grabbing all of our LangChain related packages!

In [10]:
!pip install -U -q langchain langchain-openai langchain_core langchain-community langchainhub openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 807.5/807.5 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 256.9/256.9 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.4/227.4 kB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.6/66.6 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 21.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.5/138.5 kB 15.3 MB/s eta 0:00:00


We'll also get the "star of the show" today, which is Ragas!

In [11]:
!pip install -qU ragas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.1/71.1 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.4 MB/s eta 0:00:00


As well, instead of the remote hosted solution that we used last week (Pinecone), we'll be leveraging Meta's [FAISS](https://github.com/facebookresearch/faiss) as the backend for our LangChain `VectorStore`.

We'll also install `unstructured` (from [Unstructured-IO](https://github.com/Unstructured-IO/unstructured)) and its dependencies which will allow us to load PDFs using the `UnstructuredPDFLoader` in the `langchain-community` package!

In [12]:
!pip install -qU faiss_cpu pymupdf pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 44.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 80.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 67.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.6/30.6 MB 49.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.4/345.4 kB 31.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 0.22.0 requires pandas<2.1.4,>=1.5.0, but you have pandas 2.2.1 which is incompatible.
google-colab 1.0.0 requires pandas==1.5.3, but you have pandas 2.2.1 which is incompatible.


## Task 2: Set Environment Variables

Let's set up our OpenAI API key so we can leverage their API later on.

In [13]:
import os
import openai
from getpass import getpass

openai.api_key = getpass("Please provide your OpenAI Key: ")
os.environ["OPENAI_API_KEY"] = openai.api_key

Please provide your OpenAI Key: ··········


## Task 3: Creating a Simple RAG Pipeline with LangChain v0.1.0

Building on what we learned last week, we'll be leveraging LangChain v0.1.0 and LCEL to build a simple RAG pipeline that we can baseline with Ragas.

## Building our RAG pipeline

Let's review the basic steps of RAG again:

- Create an Index
- Use retrieval to obtain pieces of context from our Index that are similar to our query
- Use a LLM to generate responses based on the retrieved context

Let's get started by creating our index.

> NOTE: We're going to start leaning on the term "index" to refer to our `VectorStore`, `VectorDatabase`, etc. We can think of "index" as the catch-all term, whereas `VectorStore` and the like relate to the specific technologies used to create, store, and interact with the index.

### Creating an Index

You'll notice that the largest changes (outside of some import changes) are that our old favourite chains are back to being bundled in an easily usable abstraction.

We can still create custom chains using LCEL - but we can also be more confident that our pre-packaged chains are creating using LCEL under the hood.

#### Loading Data

Let's start by loading some data!

> NOTE: You'll notice that we're using a document loader from the community package of LangChain. This is part of the v0.1.0 changes that make the base (`langchain-core`) package remain lightweight while still providing access to some of the more powerful community integrations.

In [15]:
!git clone https://github.com/AI-Maker-Space/DataRepository

fatal: destination path 'DataRepository' already exists and is not an empty directory.


In [16]:
from langchain_community.document_loaders import PyMuPDFLoader

loader = PyMuPDFLoader(
    "DataRepository/MuskComplaint.pdf",
)

documents = loader.load()

In [17]:
documents[0].metadata

{'source': 'DataRepository/MuskComplaint.pdf',
 'file_path': 'DataRepository/MuskComplaint.pdf',
 'page': 0,
 'total_pages': 46,
 'format': 'PDF 1.7',
 'title': '',
 'author': '',
 'subject': '',
 'keywords': '',
 'creator': '',
 'producer': '',
 'creationDate': '',
 'modDate': '',
 'trapped': ''}

#### Transforming Data

Now that we've got our single document - let's split it into smaller pieces so we can more effectively leverage it with our retrieval chain!

We'll start with the classic: `RecursiveCharacterTextSplitter`.

In [18]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 700,
    chunk_overlap = 50
)

documents = text_splitter.split_documents(documents)

Let's confirm we've split our document.

In [19]:
len(documents)

159

#### Loading OpenAI Embeddings Model

We'll need a process by which we can convert our text into vectors that allow us to compare to our query vector.

Let's use OpenAI's `text-embedding-ada-002` for this task!

In [20]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(
    model="text-embedding-ada-002"
)

#### Creating a FAISS VectorStore

Now that we have documents - we'll need a place to store them alongside their embeddings.

In [21]:
from langchain_community.vectorstores import FAISS

vector_store = FAISS.from_documents(documents, embeddings)

####❓ Question #1:

List out a few of the techniques that FAISS uses that make it performant.

> NOTE: Check the [repository](https://github.com/facebookresearch/faiss) for more information about FAISS!

In [22]:
# Answer:

"""

1) FAISS uses approaximate nearest neighbor search (ANN) by applying quantization (a technique we should know in AIM!). Basically making a small tradeoff
    in accuracy for boosting performance

2) FAISS takes advantage of vector similarities. It groups similar vectors together which helps create a faster search

3) FAISS is designed to take advantage of GPUs. This way tasks can be run in paralell and this will increase performance

4) FAISS utilizes "warp level paralellism" to maximize advantage of GPU usage. This


"""



'\n\n1) FAISS uses approaximate nearest neighbor search (ANN) by applying quantization (a technique we should know in AIM!). Basically making a small tradeoff \n    in accuracy for boosting performance\n\n2) FAISS takes advantage of vector similarities. It groups similar vectors together which helps create a faster search \n\n3) FAISS is designed to take advantage of GPUs. This way tasks can be run in paralell and this will increase performance\n\n4) FAISS utilizes "warp level paralellism" to maximize advantage of GPU usage. This \n\n\n'

#### Creating a Retriever

To complete our index, all that's left to do is expose our vectorstore as a retriever - which we can do the same way we would in previous version of LangChain!

In [23]:
retriever = vector_store.as_retriever()

#### Testing our Retriever

Now that we've gone through the trouble of creating our retriever - let's see it in action!

In [24]:
retrieved_documents = retriever.invoke("Who is the plantiff?")

In [25]:
for doc in retrieved_documents:
  print(doc)

page_content='would be owned by the foundation and used ‘for the good of the world’[.]” Plaintiff \nreplied: “Agree on all.” Ex. 2 at 1.' metadata={'source': 'DataRepository/MuskComplaint.pdf', 'file_path': 'DataRepository/MuskComplaint.pdf', 'page': 27, 'total_pages': 46, 'format': 'PDF 1.7', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator': '', 'producer': '', 'creationDate': '', 'modDate': '', 'trapped': ''}
page_content='property and derivative works funded by those monies, Plaintiff is presently unable to ascertain his \ninterest in or the use, allocation, or distribution of assets without an accounting. Plaintiff is therefore \nentitled to an accounting.' metadata={'source': 'DataRepository/MuskComplaint.pdf', 'file_path': 'DataRepository/MuskComplaint.pdf', 'page': 32, 'total_pages': 46, 'format': 'PDF 1.7', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator': '', 'producer': '', 'creationDate': '', 'modDate': '', 'trapped': ''}
page_content='1

### Creating a RAG Chain

Now that we have the "R" in RAG taken care of - let's look at creating the "AG"!

#### Creating a Prompt Template

There are a few different ways we could create our prompt template - we could create a custom template, as seen in the code below, or we could simply pull a prompt from the prompt hub! Let's look at an example of that!

In [26]:
from langchain import hub

retrieval_qa_prompt = hub.pull("langchain-ai/retrieval-qa-chat")

In [27]:
print(retrieval_qa_prompt.messages[0].prompt.template)

Answer any use questions based solely on the context below:

<context>
{context}
</context>


As you can see - the prompt template is simple (and has a small error) - so we'll create our own to be a bit more specific!

In [28]:
from langchain.prompts import ChatPromptTemplate

template = """Answer the question based only on the following context. If you cannot answer the question with the context, please respond with 'I don't know':

Context:
{context}

Question:
{question}
"""

prompt = ChatPromptTemplate.from_template(template)

#### Setting Up our Basic QA Chain

Now we can instantiate our basic RAG chain!

We'll use LCEL directly just to see an example of it - but you could just as easily use an abstraction here to achieve the same goal!

We'll also ensure to pass-through our context - which is critical for RAGAS.

In [29]:
from operator import itemgetter

from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

primary_qa_llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

retrieval_augmented_qa_chain = (
    # INVOKE CHAIN WITH: {"question" : "<<SOME USER QUESTION>>"}
    # "question" : populated by getting the value of the "question" key
    # "context"  : populated by getting the value of the "question" key and chaining it into the base_retriever
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    # "context"  : is assigned to a RunnablePassthrough object (will not be called or considered in the next step)
    #              by getting the value of the "context" key from the previous step
    | RunnablePassthrough.assign(context=itemgetter("context"))
    # "response" : the "context" and "question" values are used to format our prompt object and then piped
    #              into the LLM and stored in a key called "response"
    # "context"  : populated by getting the value of the "context" key from the previous step
    | {"response": prompt | primary_qa_llm, "context": itemgetter("context")}
)

####🏗️ Activity #1:

Describe the pipeline shown above in simple terms. You can include a diagram if desired.

In [30]:
## In short, a user will pass a question into the chain. That question is then 'piped' to the retriever. The retriever then goes and grabs relevant
## context to answer the question (based on finding embeddings closest to that of the context we passed in).

## I don't fully understand the RunnablePassthrough still, but I believe it simply carries the context forward to the next steps in the chain so it can be used
## in downstream tasks. And finally, the original question AND the new context are passed through to the LLM and a response is created

Let's test it out!

In [31]:
question = "Who is the plantiff?"

result = retrieval_augmented_qa_chain.invoke({"question" : question})

print(result["response"].content)

Elon Musk


In [32]:
question = "What does this complaint pertain to?"

result = retrieval_augmented_qa_chain.invoke({"question" : question})

print(result["response"].content)
print(result["context"])

The complaint pertains to breach of fiduciary duty, unfair business practices, accounting, and a demand for a jury trial.
[Document(page_content='1 \n2 \n3 \n4 \n5 \n6 \n7 \n8 \n9 \n10 \n11 \n12 \n13 \n14 \n15 \n16 \n17 \n18 \n19 \n20 \n21 \n22 \n23 \n24 \n25 \n26 \n27 \n28 \n \n \n– 31 – \nCOMPLAINT \n \nTHIRD CAUSE OF ACTION \nBreach of Fiduciary Duty  \nAgainst All Defendants \n133. \nPlaintiff realleges and incorporates by reference only paragraphs of this Complaint \nnecessary for his claim of Breach of Fiduciary Duty. \n134. \nUnder California law, Defendants owe fiduciary duties to Plaintiff, including a duty \nto use Plaintiff’s contributions for the purposes for which they were made. E.g., Cal. Bus. & Prof. \nCode § 17510.8. Defendants have repeatedly breached their fiduciary duties to Plaintiff, including \nby:', metadata={'source': 'DataRepository/MuskComplaint.pdf', 'file_path': 'DataRepository/MuskComplaint.pdf', 'page': 30, 'total_pages': 46, 'format': 'PDF 1.7', 'title':

We can already see that there are some improvements we could make here.

For now, let's switch gears to RAGAS to see how we can leverage that tool to provide us insight into how our pipeline is performing!

# 🤝 Breakout Room #2

## Task 1: Synthetic Dataset Generation for Evaluation using Ragas

Ragas is a powerful library that lets us evaluate our RAG pipeline by collecting input/output/context triplets and obtaining metrics relating to a number of different aspects of our RAG pipeline.

We'll be evluating on every core metric today, but in order to do that - we'll need to creat a test set. Luckily for us, Ragas can do that directly!

### Synthetic Test Set Generation

We can leverage Ragas' [`Synthetic Test Data generation`](https://docs.ragas.io/en/stable/concepts/testset_generation.html) functionality to generate our own synthetic QC pairs - as well as a synthetic ground truth - quite easily!

> NOTE: This process will use `gpt-3.5-turbo-16k` as the base generator and `gpt-4` as the critic - if you're attempting to create a lot of samples please be aware of cost, as well as rate limits.

In [33]:
eval_documents = documents

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1500,
    chunk_overlap = 400
)

eval_documents = text_splitter.split_documents(eval_documents)

####❓ Question #2:

Why is it important to split our documents using different parameters when creating our synthetic data?

In [34]:
## Answer:
"""
  This is necessary so we don't inadvertantly bias the 'ground truth' model (the synthetic data) in our favor. Using different sized chunks will
  create a more unique document structure for our ground truth synthetic data which will be a more 'realistic' example of what the model is likely to
  encounter in production.

  On a similar note, this should also help to ensure our models are generalizing well. That is really the key is the our models will generalize well on unseen
  data when in production. Using different chunking strategies is necessary to ensure our models find the right general themes within the data.

  It can also help you explore trade-offs between retrieval efficiency and quality of context retrieved.

"""

"\n  This is necessary so we don't inadvertantly bias the 'ground truth' model (the synthetic data) in our favor. Using different sized chunks will\n  create a more unique document structure for our ground truth synthetic data which will be a more 'realistic' example of what the model is likely to\n  encounter in production.\n\n  On a similar note, this should also help to ensure our models are generalizing well. That is really the key is the our models will generalize well on unseen\n  data when in production. Using different chunking strategies is necessary to ensure our models find the right general themes within the data.\n\n  It can also help you explore trade-offs between retrieval efficiency and quality of context retrieved.\n\n"

In [35]:
len(documents)

159

In [36]:
from ragas.testset.generator import TestsetGenerator
from ragas.testset.evolutions import simple, reasoning, multi_context

generator = TestsetGenerator.with_openai()

testset = generator.generate_with_langchain_docs(documents, test_size=10, distributions={simple: 0.25, reasoning: 0.25, multi_context: 0.5})

embedding nodes:   0%|          | 0/318 [00:00<?, ?it/s]

Generating:   0%|          | 0/10 [00:00<?, ?it/s]

####❓ Question #3:

`{simple: 0.5, reasoning: 0.25, multi_context: 0.25}`

What exactly does this mapping refer to?

> NOTE: Check out the Ragas documentation on this generation process [here](https://docs.ragas.io/en/stable/concepts/testset_generation.html).

In [ ]:
# Answer:

"""
  This is effectively a breakdown of the **types** of questions you want to generate with synthetic data. RAGAS offers these options
  where simple is a more basic question, reasoning rewrites the question to enhance the need for reasoning to answer effectively, and multi context
  is written in a way to require context from multiple chunks to answer.

  They also offer conditioning and conversational questions

"""

Let's look at the output and see what we can learn about it!

In [38]:
testset.test_data[0]

DataRow(question='What is the hope behind releasing the full, open version of future powerful models?', contexts=['challenging.” At the time, OpenAI stated that it was releasing the full, open version with the hope \nthat it “will be useful to developers of future powerful models.” This release was accompanied by a \ndetailed paper co-authored by OpenAI scientists as well as independent social and technical \nscientists. This paper explained just some of the many benefits that came from releasing models \npublically as opposed to keeping them closed.'], ground_truth='The hope behind releasing the full, open version of future powerful models is that it will be useful to developers of future powerful models.', evolution_type='simple')

### Generating Responses with RAG Pipeline

Now that we have some QC pairs, and some ground truths, let's evaluate our RAG pipeline using Ragas.

The process is, again, quite straightforward - thanks to Ragas and LangChain!

Let's start by extracting our questions and ground truths from our create testset.

We can start by converting our test dataset into a Pandas DataFrame.

In [39]:
test_df = testset.to_pandas()

In [40]:
test_df

,question,contexts,ground_truth,evolution_type,episode_done
0,"What is the hope behind releasing the full, op...","[challenging.” At the time, OpenAI stated that...","The hope behind releasing the full, open versi...",simple,True
1,How did OpenAI use reinforcement learning in t...,[77. \nInitial work at OpenAI followed much in...,OpenAI used reinforcement learning to play Dot...,simple,True
2,How did researchers at the University of Tokyo...,[implementation for others to build on. \n84. ...,Researchers at the University of Tokyo and Goo...,reasoning,True
3,Which strategy video game did OpenAI surpass a...,[77. \nInitial work at OpenAI followed much in...,OpenAI surpassed a world-champion program in t...,reasoning,True
4,"""What was the impact of OpenAI's GPT-3 on futu...",[1 \n2 \n3 \n4 \n5 \n6 \n7 \n8 \n9 \n10 \n11 \...,Their publication did prove to be useful to th...,multi_context,True
5,"""What game did OpenAI use reinforcement learni...",[77. \nInitial work at OpenAI followed much in...,OpenAI used reinforcement learning to beat a w...,multi_context,True
6,"""What strategy video game did OpenAI excel in,...",[77. \nInitial work at OpenAI followed much in...,OpenAI excelled in the strategy video game Dot...,multi_context,True
7,What concerns arise from the potential investm...,[chip fabrication plants. If $10 billion from ...,The concerns arise from the potential investme...,multi_context,True
8,What concerns about AGI were raised by Bill Jo...,[18. \nMr. Musk has long recognized that AGI p...,The concerns about AGI raised by Bill Joy and ...,multi_context,True
9,"What happened to OpenAI's ""key documents"" in r...",[proceeds going to another charitable enterpri...,OpenAI has shuttered the public's access to pr...,reasoning,True


In [41]:
test_questions = test_df["question"].values.tolist()
test_groundtruths = test_df["ground_truth"].values.tolist()

Now we'll generate responses using our RAG pipeline using the questions we've generated - we'll also need to collect our retrieved contexts for each question.

We'll do this in a simple loop to see exactly what's happening!

In [42]:
answers = []
contexts = []

for question in test_questions:
  response = retrieval_augmented_qa_chain.invoke({"question" : question})
  answers.append(response["response"].content)
  contexts.append([context.page_content for context in response["context"]])

Now we can wrap our information in a Hugging Face dataset for use in the Ragas library.

In [43]:
from datasets import Dataset

response_dataset = Dataset.from_dict({
    "question" : test_questions,
    "answer" : answers,
    "contexts" : contexts,
    "ground_truth" : test_groundtruths
})

Let's take a peek and see what that looks like!

In [44]:
response_dataset[0]

{'question': 'What is the hope behind releasing the full, open version of future powerful models?',
 'answer': 'The hope behind releasing the full, open version of future powerful models is that it will be useful to developers.',
 'contexts': ['challenging.” At the time, OpenAI stated that it was releasing the full, open version with the hope \nthat it “will be useful to developers of future powerful models.” This release was accompanied by a \ndetailed paper co-authored by OpenAI scientists as well as independent social and technical \nscientists. This paper explained just some of the many benefits that came from releasing models \npublically as opposed to keeping them closed.',
  'which needed to be trained on a specific task, “[w]hen a large language model is trained on a \nsufficiently large and diverse dataset it is able to perform well across many domains and datasets.” \nThese models were proving themselves to be very different from previous AI systems. Instead of \ntraining the

## Task 2: Evaluating our Pipeline with Ragas

Now that we have our response dataset - we can finally get into the "meat" of Ragas - evaluation!

First, we'll import the desired metrics, then we can use them to evaluate our created dataset!

Check out the specific metrics we'll be using in the Ragas documentation:

- [Faithfulness](https://docs.ragas.io/en/stable/concepts/metrics/faithfulness.html)
- [Answer Relevancy](https://docs.ragas.io/en/stable/concepts/metrics/answer_relevance.html)
- [Context Precision](https://docs.ragas.io/en/stable/concepts/metrics/context_precision.html)
- [Context Recall](https://docs.ragas.io/en/stable/concepts/metrics/context_recall.html)
- [Answer Correctness](https://docs.ragas.io/en/stable/concepts/metrics/answer_correctness.html)

See the accompanied presentation for more in-depth explanations about each of the metrics!

In [45]:
from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    answer_correctness,
    context_recall,
    context_precision,
)

metrics = [
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision,
    answer_correctness,
]

All that's left to do is call "evaluate" and away we go!

In [46]:
results = evaluate(response_dataset, metrics)

Evaluating:   0%|          | 0/50 [00:00<?, ?it/s]

In [47]:
results

{'faithfulness': 0.7778, 'answer_relevancy': 0.9381, 'context_recall': 0.9667, 'context_precision': 0.9056, 'answer_correctness': 0.8247}

In [48]:
results_df = results.to_pandas()
results_df

,question,answer,contexts,ground_truth,faithfulness,answer_relevancy,context_recall,context_precision,answer_correctness
0,"What is the hope behind releasing the full, op...","The hope behind releasing the full, open versi...","[challenging.” At the time, OpenAI stated that...","The hope behind releasing the full, open versi...",1.0,0.989220,1.000000,1.000000,0.748455
1,How did OpenAI use reinforcement learning in t...,OpenAI used reinforcement learning to compete ...,[77. \nInitial work at OpenAI followed much in...,OpenAI used reinforcement learning to play Dot...,1.0,0.971244,1.000000,1.000000,0.746960
2,How did researchers at the University of Tokyo...,Researchers at the University of Tokyo and Goo...,[implementation for others to build on. \n84. ...,Researchers at the University of Tokyo and Goo...,1.0,0.947469,1.000000,1.000000,0.740529
3,Which strategy video game did OpenAI surpass a...,Dota 2,[77. \nInitial work at OpenAI followed much in...,OpenAI surpassed a world-champion program in t...,1.0,0.913245,1.000000,1.000000,0.964171
4,"""What was the impact of OpenAI's GPT-3 on futu...",The impact of OpenAI's GPT-3 on future model d...,[1 \n2 \n3 \n4 \n5 \n6 \n7 \n8 \n9 \n10 \n11 \...,Their publication did prove to be useful to th...,NaN,0.955423,1.000000,0.805556,0.731544
5,"""What game did OpenAI use reinforcement learni...",Dota 2,[77. \nInitial work at OpenAI followed much in...,OpenAI used reinforcement learning to beat a w...,0.0,0.926096,0.666667,0.750000,0.961317
6,"""What strategy video game did OpenAI excel in,...",Dota 2,[a superhuman level of play in the games of ch...,OpenAI excelled in the strategy video game Dot...,0.0,0.888023,1.000000,0.500000,0.968325
7,What concerns arise from the potential investm...,The concerns arise from the potential influenc...,[the Board’s ability to control Mr. Altman’s u...,The concerns arise from the potential investme...,1.0,0.920141,1.000000,1.000000,0.791998
8,What concerns about AGI were raised by Bill Jo...,The concerns raised by Bill Joy and echoed by ...,[18. \nMr. Musk has long recognized that AGI p...,The concerns about AGI raised by Bill Joy and ...,1.0,0.959949,1.000000,1.000000,0.844706
9,"What happened to OpenAI's ""key documents"" in r...",OpenAI has recently shuttered the public's acc...,[proceeds going to another charitable enterpri...,OpenAI has shuttered the public's access to pr...,1.0,0.909965,1.000000,1.000000,0.749080


## Task 3: Making Adjustments to our RAG Pipeline

Now that we have established a baseline - we can see how any changes impact our pipeline's performance!

Let's modify our retriever and see how that impacts our Ragas metrics!

In [49]:
from langchain.retrievers import MultiQueryRetriever

advanced_retriever = MultiQueryRetriever.from_llm(retriever=retriever, llm=primary_qa_llm)

We'll also re-create our RAG pipeline using the abstractions that come packaged with LangChain v0.1.0!

First, let's create a chain to "stuff" our documents into our context!

In [50]:
from langchain.chains.combine_documents import create_stuff_documents_chain

document_chain = create_stuff_documents_chain(primary_qa_llm, retrieval_qa_prompt)

Next, we'll create the retrieval chain!

In [51]:
from langchain.chains import create_retrieval_chain

retrieval_chain = create_retrieval_chain(advanced_retriever, document_chain)

In [52]:
response = retrieval_chain.invoke({"input": "Who is the plantiff?"})

In [53]:
print(response["answer"])

The plaintiff is Elon Musk.


In [54]:
response = retrieval_chain.invoke({"input": "What does this complaint pertain to?"})

In [55]:
print(response["answer"])

The complaint pertains to a legal case involving Plaintiff Elon Musk alleging breach of fiduciary duty, unfair business practices, and seeking accounting, restitution, disgorgement of monies received by Defendants, prejudgment interest, injunction against future activities, and specific performance. The complaint also includes a demand for a jury trial.


Well, just from those responses this chain *feels* better - but lets see how it performs on our eval!

Let's do the same process we did before to collect our pipeline's contexts and answers.

In [56]:
answers = []
contexts = []

for question in test_questions:
  response = retrieval_chain.invoke({"input" : question})
  answers.append(response["answer"])
  contexts.append([context.page_content for context in response["context"]])

Now we can convert this into a dataset, just like we did before.

In [57]:
response_dataset_advanced_retrieval = Dataset.from_dict({
    "question" : test_questions,
    "answer" : answers,
    "contexts" : contexts,
    "ground_truth" : test_groundtruths
})

Let's evaluate on the same metrics we did for the first pipeline and see how it does!

In [58]:
advanced_retrieval_results = evaluate(response_dataset_advanced_retrieval, metrics)

Evaluating:   0%|          | 0/50 [00:00<?, ?it/s]

In [59]:
advanced_retrieval_results_df = advanced_retrieval_results.to_pandas()
advanced_retrieval_results_df

,question,answer,contexts,ground_truth,faithfulness,answer_relevancy,context_recall,context_precision,answer_correctness
0,"What is the hope behind releasing the full, op...","The hope behind releasing the full, open versi...","[challenging.” At the time, OpenAI stated that...","The hope behind releasing the full, open versi...",1.000000,0.989225,1.0,1.000000,0.747840
1,How did OpenAI use reinforcement learning in t...,OpenAI used reinforcement learning to compete ...,[77. \nInitial work at OpenAI followed much in...,OpenAI used reinforcement learning to play Dot...,0.666667,0.872291,0.5,1.000000,0.540233
2,How did researchers at the University of Tokyo...,Researchers at the University of Tokyo and Goo...,[implementation for others to build on. \n84. ...,Researchers at the University of Tokyo and Goo...,0.500000,0.949140,1.0,1.000000,0.618118
3,Which strategy video game did OpenAI surpass a...,OpenAI surpassed a world-champion program in D...,[77. \nInitial work at OpenAI followed much in...,OpenAI surpassed a world-champion program in t...,1.000000,0.966231,1.0,1.000000,0.742693
4,"""What was the impact of OpenAI's GPT-3 on futu...",The publication of OpenAI's GPT-3 proved to be...,[1 \n2 \n3 \n4 \n5 \n6 \n7 \n8 \n9 \n10 \n11 \...,Their publication did prove to be useful to th...,NaN,0.956064,1.0,0.804167,0.738901
5,"""What game did OpenAI use reinforcement learni...",OpenAI used reinforcement learning to beat a w...,[77. \nInitial work at OpenAI followed much in...,OpenAI used reinforcement learning to beat a w...,NaN,0.906220,1.0,0.700000,0.999786
6,"""What strategy video game did OpenAI excel in,...","OpenAI excelled in Dota 2, a strategy video ga...",[77. \nInitial work at OpenAI followed much in...,OpenAI excelled in the strategy video game Dot...,1.000000,0.907280,1.0,1.000000,0.740209
7,What concerns arise from the potential investm...,"The potential investments in OpenAI, Inc. rais...",[chip fabrication plants. If $10 billion from ...,The concerns arise from the potential investme...,1.000000,0.916848,1.0,1.000000,0.690466
8,What concerns about AGI were raised by Bill Jo...,Bill Joy and Elon Musk both raised concerns ab...,[18. \nMr. Musk has long recognized that AGI p...,The concerns about AGI raised by Bill Joy and ...,1.000000,0.850803,1.0,0.833333,0.646986
9,"What happened to OpenAI's ""key documents"" in r...",OpenAI has recently shuttered the public's acc...,[proceeds going to another charitable enterpri...,OpenAI has shuttered the public's access to pr...,1.000000,0.000000,1.0,0.733333,0.530397


## Task 4: Evaluating our Adjusted Pipeline Against Our Baseline

Now we can compare our results and see what directional changes occured!

Let's refresh with our initial metrics.

In [60]:
results

{'faithfulness': 0.7778, 'answer_relevancy': 0.9381, 'context_recall': 0.9667, 'context_precision': 0.9056, 'answer_correctness': 0.8247}

And see how our advanced retrieval modified our chain!

In [61]:
advanced_retrieval_results

{'faithfulness': 0.8958, 'answer_relevancy': 0.8314, 'context_recall': 0.9500, 'context_precision': 0.9071, 'answer_correctness': 0.6996}

In [62]:
import pandas as pd

df_original = pd.DataFrame(list(results.items()), columns=['Metric', 'Baseline'])
df_comparison = pd.DataFrame(list(advanced_retrieval_results.items()), columns=['Metric', 'MultiQueryRetriever with Document Stuffing'])

df_merged = pd.merge(df_original, df_comparison, on='Metric')

df_merged['Delta'] = df_merged['MultiQueryRetriever with Document Stuffing'] - df_merged['Baseline']

df_merged

,Metric,Baseline,MultiQueryRetriever with Document Stuffing,Delta
0,faithfulness,0.777778,0.895833,0.118056
1,answer_relevancy,0.938078,0.831410,-0.106667
2,context_recall,0.966667,0.950000,-0.016667
3,context_precision,0.905556,0.907083,0.001528
4,answer_correctness,0.824708,0.699563,-0.125146


## Task 5: Testing OpenAI's Claim

Now that we've seen how our retriever can impact the performance of our RAG pipeline - let's see how changing our embedding model impacts performance.

####🏗️ Activity #2:

Please provide markdown, or code comments, to explain which each of the following steps are doing!

In [63]:
### loading in our embedding model - we will use text-embedding-3-small from openAI

In [64]:
new_embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

In [65]:
### loading in our vector database. For this task, we will use FAISS. This is a vector store from facebook which is open sourced :)
### this vector store will enable our RAG
### we will also pass through our embeddings model created above

In [66]:
vector_store = FAISS.from_documents(documents, new_embeddings)

In [67]:
### now we will create a retriever object from our vector store created in the step above

In [68]:
new_retriever = vector_store.as_retriever()

In [69]:
### with this stpe below, we will create a more advanced retriever which can handle multiple user queries.
### So if a user asks: Who is Tiger Woods and what are some of his main accomplishments it could parse this single query into 2 distinct queries and
### retrieve separate embeddings from our context documents to best answer each question

### This should enhance retrieval capabilities by using strenghts of the retriever and the LLM

In [70]:
new_advanced_retriever = MultiQueryRetriever.from_llm(retriever=new_retriever, llm=primary_qa_llm)

In [71]:
## and finally, we will create our new chain by chaining together our advanced retriver and document chain. This combines:

### Advanced retriever for multi-query chat using `primary_qa_llm`.
### Tailored document processing from `create_stuff_documents_chain` with the same model and prompt.

In [72]:
new_retrieval_chain = create_retrieval_chain(new_advanced_retriever, document_chain)

In [73]:
### in this code, we first create empty lists for answers and contexts, then we create a loop where
### fowe pass each question from our test_questions list through the new_retieval_chain to generate new answers

### for additional context we are aiming to see if implementing the multiquery retriever improves our eval metrics for our RAG model

In [75]:
answers = []
contexts = []

for question in test_questions:
  response = new_retrieval_chain.invoke({"input" : question})
  answers.append(response["answer"])
  contexts.append([context.page_content for context in response["context"]])

In [76]:
## Herewe are getting our data sets (questions, asnwer, contexts, and ground truth) into a nice format which is required by
### the RAGAS package. We need to do this so we can easily use RAGAS for quick eval (next step)

In [77]:
new_response_dataset_advanced_retrieval = Dataset.from_dict({
    "question" : test_questions,
    "answer" : answers,
    "contexts" : contexts,
    "ground_truth" : test_groundtruths
})

In [78]:
### apllying RAGAS to our asnwers from the MultiQueryRetriever chain we created above. This will tell us whether or not the mutli query retreiver is
### adding any value

In [79]:
new_advanced_retrieval_results = evaluate(new_response_dataset_advanced_retrieval, metrics)

Evaluating:   0%|          | 0/50 [00:00<?, ?it/s]

In [80]:
### printing RAGAS results

In [81]:
new_advanced_retrieval_results

{'faithfulness': 0.9444, 'answer_relevancy': 0.8470, 'context_recall': 1.0000, 'context_precision': 0.9258, 'answer_correctness': 0.7053}

In [82]:
### creating a dataframe to compare the RAGAS results from multiple different models

In [83]:
df_baseline = pd.DataFrame(list(results.items()), columns=['Metric', 'Baseline'])
df_original = pd.DataFrame(list(advanced_retrieval_results.items()), columns=['Metric', 'ADA'])
df_comparison = pd.DataFrame(list(new_advanced_retrieval_results.items()), columns=['Metric', 'Text Embedding 3'])

df_merged = pd.merge(df_original, df_comparison, on='Metric')
df_merged = pd.merge(df_baseline, df_merged, on="Metric")

df_merged['Delta - TE3 -> ADA'] = df_merged['Text Embedding 3'] - df_merged['ADA']
df_merged['Delta - TE3 -> Baseline'] = df_merged['Text Embedding 3'] - df_merged['Baseline']

df_merged

,Metric,Baseline,ADA,Text Embedding 3,Delta - TE3 -> ADA,Delta - TE3 -> Baseline
0,faithfulness,0.777778,0.895833,0.944444,0.048611,0.166667
1,answer_relevancy,0.938078,0.831410,0.846996,0.015586,-0.091082
2,context_recall,0.966667,0.950000,1.000000,0.050000,0.033333
3,context_precision,0.905556,0.907083,0.925813,0.018730,0.020258
4,answer_correctness,0.824708,0.699563,0.705288,0.005725,-0.119420


####❓ Question #4:

Do you think, in your opinion, `text-embedding-3-small` is significantly better than `ada`?

In [86]:
# Yes, it does seem to be better on most metrics. There were notable improvements on faithfulness and context recall

## Very minor improvements on answer relevancy, context precision, and answer correctness

## Overall you could make a case that text embedding 3 is slightly better but of course to truly draw
## that conclusion I would need much more data than 10 samples. Very cool process though!

## BONUS ACTIVITY: Showcase Multi-Context Perfomance Changes

Now that we've looked at a number of different examples - showcase the difference on the multi-context *specific* questions that were synthetically generated.

> NOTE: You have all the data you'll need already in the notebook if you made it to this step!

In [ ]:
### YOUR CODE HERE